<a href="https://colab.research.google.com/github/JK-the-Ko/Thermo-Fluid-Dynamics-Experiment/blob/main/2023-2/%EC%97%B4%EC%9C%A0%EC%B2%B4%EA%B3%B5%ED%95%99%EC%8B%A4%ED%97%981_Week9_%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network

## Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Create Function

In [ ]:
def function(x1, x2, x3) :
  return np.power(x1,2) + 4*np.power(x2,2) - 10*x3

In [ ]:
deltaX = 1e-3
x1, x2, x3 = np.arange(2, 6, deltaX), np.arange(-4, 0, deltaX), np.arange(0, 4, deltaX)

In [ ]:
y = function(x1, x2, x3)

### Visualize Histogram

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(y, bins = 100)
plt.xlabel("value")
plt.ylabel("frequency")
plt.title("Output Distribution")
plt.show()

## Activation Function

In [ ]:
np.random.seed(42)

### Sigmoid

In [ ]:
def Sigmoid(input:np.array)->np.array :
  return np.power(1 + np.exp(-input), -1)

#### Random Sampling

In [ ]:
numSample = 25

In [ ]:
randomVariables = np.random.randn((numSample))

#### Compare Result

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(np.arange(numSample), randomVariables, label="Original", marker="X", s=250)
plt.scatter(np.arange(numSample), Sigmoid(randomVariables), label="Sigmoid", marker=".", s=250)

for i in range(numSample) :
  deltaY = Sigmoid(randomVariables[i]) - randomVariables[i]
  if deltaY != 0 :
    plt.arrow(i, randomVariables[i], 0, deltaY, head_width = 0.25, head_length = 0.05, fc = "k", ec = "k")

plt.xlabel("# sample")
plt.ylabel("value")
plt.title("Result Comparison (Sigmoid)")
plt.legend(loc="best")
plt.show()

### Rectified Linear Unit (ReLU)

In [ ]:
def ReLU(input:np.array)->np.array :
  return np.where(input > 0, input, 0)

#### Random Sampling

In [ ]:
numSample = 25

In [ ]:
randomVariables = np.random.randn((numSample))

#### Compare Result

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(np.arange(numSample), randomVariables, label="Original", marker="X", s=250)
plt.scatter(np.arange(numSample), ReLU(randomVariables), label="ReLU", marker=".", s=250)

for i in range(numSample) :
  deltaY = ReLU(randomVariables[i]) - randomVariables[i]
  if deltaY != 0 :
    plt.arrow(i, randomVariables[i], 0, deltaY, head_width = 0.25, head_length = 0.05, fc = "k", ec = "k")

plt.xlabel("# sample")
plt.ylabel("value")
plt.title("Result Comparison (ReLU)")
plt.legend(loc="best")
plt.show()

## Build 2-Layer Neural Network

In [ ]:
from tqdm import tqdm

In [ ]:
class TwoLayerNeuralNetwork :
  def __init__(self, numInputs:int, numHiddenLayerNodes:int, numOutputs:int, seed:int) :
    # Initialize Variables
    self.numInputs = numInputs
    self.numHiddenLayerNodes = numHiddenLayerNodes
    self.numOutputs = numOutputs
    self.seed = seed

    # Initialize Model Parameters
    self.initializeWeights()

  def initializeWeights(self) :
    # Fix Seed
    np.random.seed(self.seed)

    # Initialize Model Parameters
    self.layer1Weights = np.random.random((self.numInputs, self.numHiddenLayerNodes))
    self.layer2Weights = np.random.random((self.numHiddenLayerNodes, self.numOutputs))

    # Print Model Parameters
    print("Model Parameters Initialized!")
    print(f"# Parameters : {self.layer1Weights.size + self.layer2Weights.size}")
    print(f"Layer 1 Size : {self.layer1Weights.shape}")
    print(f"Layer 1 Weights: {self.layer1Weights.flatten()}")
    print(f"Layer 2 Size : {self.layer2Weights.shape}")
    print(f"Layer 2 Weights: {self.layer2Weights.flatten()}")

  def LeakyReLU(self, input:np.array)->np.array :
    return np.where(input>0, input, -0.2*input)

  def predict(self, input:np.array)->np.array :
    output = self.LeakyReLU(np.matmul(input, self.layer1Weights))
    output = np.matmul(output, self.layer2Weights)
    return output

  def computeMSELoss(self, pred:np.array, target:np.array)->float :
    return np.power(target-pred, 2).mean()

  def backPropagation(self, input:np.array, target:np.array)->np.array :
    # Compute Each Layer Output
    stg1Output = np.matmul(input, self.layer1Weights)
    stg2Output = self.LeakyReLU(stg1Output)
    stg3Output = np.matmul(stg2Output, self.layer2Weights)

    # Compute Gradient of Each Parameter
    gradientLayer2 = -np.matmul(stg2Output.reshape(-1,1), (target-stg3Output).reshape(1,-1))
    gradientLayer1 = -np.matmul(input.reshape(-1,1), (target-stg3Output).mean() * (self.layer2Weights * np.where(stg2Output>0, 1, -0.2)).sum(axis=1).reshape(1,-1))

    return gradientLayer1, gradientLayer2

  def train(self, inputTrain:np.array, targetTrain:np.array, inputTest:np.array, targetTest:np.array,batchSize:int, learningRate:float)->list :
    # Create List Instance
    trainLossList, testLossList = [],[]

    # Initialize Varaibles
    loss = 0

    # Compute Iteration
    iteration = len(inputTrain) // batchSize

    print("Training Phase")
    with tqdm(total = iteration) as pBar :
      for i in range(iteration) :
        # Initialize Varaibles
        gradientLayer1, gradientLayer2 = 0, 0
        subInputTrain, subTargetTrain = inputTrain[i*batchSize : (i+1)*batchSize], targetTrain[i*batchSize : (i+1)*batchSize]

        for j in range(batchSize) :
          # Feed Forward
          pred = self.predict(subInputTrain[j])

          # Compute MSE Loss
          loss += self.computeMSELoss(pred, subTargetTrain[j])

          # Compute Gradient of Each Data
          subGradientLayer1, subGradientLayer2 = self.backPropagation(subInputTrain[j], subTargetTrain[j])
          gradientLayer1 += subGradientLayer1
          gradientLayer2 += subGradientLayer2

        # Compute Average Gradient
        gradientLayer1 /= batchSize
        gradientLayer2 /= batchSize

        # Update Model Parameters
        self.layer1Weights -= learningRate * gradientLayer1
        self.layer2Weights -= learningRate * gradientLayer2

        # Update TQDM Bar
        pBar.update()

    # Compute Average Loss
    loss /= len(inputTrain)
    trainLossList.append(loss)

    # Initialize Varaibles
    loss = 0

    # Compute Iteration
    iteration = len(inputTest) // batchSize

    print("Test Phase")
    with tqdm(total = iteration) as pBar :
      for i in range(iteration) :
        # Initialize Varaibles
        subInputTest, subTargetTest = inputTest[i*batchSize : (i+1)*batchSize], targetTest[i*batchSize : (i+1)*batchSize]

        for j in range(batchSize) :
          # Feed Forward
          pred = self.predict(subInputTest[j])

          # Compute MSE Loss
          loss += self.computeMSELoss(pred, subTargetTest[j])

        # Update TQDM Bar
        pBar.update()

     # Compute Average Loss
      loss /= len(inputTest)
      testLossList.append(loss)

    return trainLossList, testLossList

## Generate Dataset

In [ ]:
inputDataset, targetDataset = [], []

for i in range(len(y)) :
  inputDataset.append(np.array([x1[i], x2[i], x3[i]]))
  targetDataset.append(np.array(y[i]))

inputDataset, targetDataset = np.array(inputDataset), np.array(targetDataset)

In [ ]:
print(f"Input Dataset Size : {inputDataset.shape}")
print(f"Target Dataset Size : {targetDataset.shape}")

### Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(inputDataset, targetDataset, test_size = 0.2, random_state = 42)

In [ ]:
print(f"Training Dataset Size : {xTrain.shape[0]}")
print(f"Test Dataset Size : {xTest.shape[0]}")

## Create Neural Network Instance

In [ ]:
numInputs, numHiddenLayerNodes, numOutputs, seed = 3, 2, 1, 42

In [ ]:
model = TwoLayerNeuralNetwork(numInputs, numHiddenLayerNodes, numOutputs, seed)

## Determine Hyperparameters

In [ ]:
numEpoch = 200

In [ ]:
batchSize, learningRate = 128, 1e-4

## Train Model

In [ ]:
def trainModel(model, numEpoch, batchSize, learningRate) :
  trainLossList, testLossList = [],[]
  bestLoss = np.inf

  for epoch in range(numEpoch) :
    print(f"[Current Epoch : {epoch + 1}]")
    trainLoss, testLoss = model.train(xTrain, yTrain, xTest, yTest, batchSize, learningRate)
    trainLossList += trainLoss
    testLossList += testLoss

    if testLoss[0] < bestLoss :
      bestLoss = testLoss[0]
      bestWeight = [model.layer1Weights, model.layer2Weights]

  return trainLossList, testLossList, bestWeight

In [ ]:
trainLossList, testLossList, bestWeight = trainModel(model, numEpoch, batchSize, learningRate)

## Plot Loss Curve

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.arange(len(trainLossList)), trainLossList, label = "Train Loss")
plt.plot(np.arange(len(testLossList)), testLossList, label = "Test Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training Result")
plt.legend(loc = "best")
plt.show()

## Get Optimized Parameters

In [ ]:
min(testLossList)

In [ ]:
print(bestWeight[0])
print(bestWeight[1])

## Inference Result with Trained Neural Network

In [ ]:
model.layer1Weights, model.layer2Weights = bestWeight[0], bestWeight[1]

In [ ]:
yPredNN = []

for subXTest in xTest :
  yPredNN.append(model.predict(subXTest)[0])

## Inference Result with Linear Regression Model

In [ ]:
def getParameter(x: np.array, y: np.array) :
  xT = np.transpose(x)
  output = np.matmul(np.matmul(np.linalg.inv(np.matmul(xT, x)), xT), y)

  return output

In [ ]:
betaHat = getParameter(xTrain, yTrain)

In [ ]:
betaHat

In [ ]:
yPredLR = np.matmul(xTest, betaHat).tolist()

## Compare Result

### Compute MSE Loss

In [ ]:
def MSELoss(yPred:np.array, yTrue:np.array)->float:
  return np.power(yPred-yTrue, 2).mean()

In [ ]:
print(f"Neural Network (NN) MSE Loss : {MSELoss(np.array(yPredNN), yTest)}")
print(f"Linear Regression (LR) MSE Loss : {MSELoss(np.array(yPredLR), yTest)}")

### Plot Bar Chart

In [ ]:
fig, ax = plt.subplots(figsize = (20, 10))
idx = np.asarray([i for i in range(50)])
width = 0.2

ax.bar(idx, yTest[:50], width = width)
ax.bar(idx+width, yPredNN[:50], width = width)
ax.bar(idx+2*width, yPredLR[:50], width = width)
ax.set_xticks(idx)
ax.legend(["Ground Truth", "NN", "LR"])
ax.set_xlabel("# samples")
ax.set_ylabel("Value")
ax.set_title("Result Comparison")

fig.tight_layout()
plt.show()